# 관측 가능성을 갖춘 Bedrock AgentCore Runtime에서의 Strands Agent 배포

이 워크샵은 Amazon Bedrock AgentCore Runtime을 사용하여 Strands Agent를 배포하는 방법을 보여주며, 

포괄적인 AI 에이전트 기능을 위해 내장 도구와 사용자 정의 함수를 통합합니다.

## 개요

이 실습에서는 다음을 수행합니다:
- 도구가 포함된 Strands Agent를 Bedrock AgentCore Runtime에 배포
- IAM 권한과 함께 `boto3`를 사용하여 배포된 에이전트 호출
- Bedrock AgentCore Runtime 세션의 특성 이해
- GenAI 관찰 가능성 및 추적성에 대한 학습

## 사전 요구사항

이 실습을 시작하기 전에 다음이 준비되어 있는지 확인하세요:
- AWS 자격 증명 구성 (IAM 역할 또는 환경 변수)
- 필요한 Python 패키지 설치
- AWS 리전에 따른 Nova Pro 모델 ID
- Bedrock AgentCore 관찰 가능성을 위한 CloudWatch Transaction Search 활성화

IAM 역할이 가정된 환경에서 실행하지 않는 경우, AWS 자격 증명을 환경 변수로 설정하세요:

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

Strands Agent와 Bedrock AgentCore SDK에 필요한 패키지를 설치합니다:

In [ ]:
#%pip install -q strands-agents strands-agents-tools bedrock-agentcore bedrock-agentcore-starter-toolkit

### 1단계: 에이전트 구성이 포함된 Python 파일 생성

내장 및 사용자 정의 도구가 포함된 Strands Agent를 정의하는 메인 Python 파일을 생성합니다. 이 파일은 컨테이너화된 서비스로 Bedrock AgentCore Runtime에 배포됩니다.

**배포 요구사항:**
AgentCore Runtime에 에이전트를 배포하려면 다음이 필요합니다:
- Runtime App 가져오기: `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
- App 초기화: `app = BedrockAgentCoreApp()`
- 호출 함수를 `@app.entrypoint`로 데코레이트
- `app.run()`으로 AgentCore Runtime이 실행을 제어하도록 허용

In [ ]:
%%writefile strands_agent.py
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import calculator
import boto3
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

# AWS 리전에 따라 Nova Pro 모델 ID 설정
NOVA_PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
region = boto3.session.Session().region_name
if region.startswith("eu"):
    NOVA_PRO_MODEL_ID = "eu.amazon.nova-pro-v1:0"
elif region.startswith("ap"):
    NOVA_PRO_MODEL_ID = "apac.amazon.nova-pro-v1:0"

@tool
def weather(city: str) -> str:
    """도시의 날씨 정보를 가져옵니다
    Args:
        city: 도시 또는 위치 이름
    """
    return f"{city}의 날씨: 맑음, 35°C"


# 포괄적인 Strands Agent 생성 및 테스트
agent = Agent(
    model=BedrockModel(model_id=NOVA_PRO_MODEL_ID),
    system_prompt = """당신은 간결한 응답을 제공하는 도움이 되는 어시스턴트입니다.
                    """,
    tools=[weather, calculator],
)

@app.entrypoint
async def strands_agent_bedrock(payload, context):
    """
    페이로드로 에이전트를 호출합니다
    """
    print(f"Payload: {payload}")
    print(f"Context: {context}")
    user_input = payload.get("prompt", "프롬프트를 찾을 수 없습니다")
    response = agent(user_input)
    return response
    
    # 스트리밍 모드
    """
    stream = agent.stream_async(user_input)
    async for event in stream:
        if "data" in event:
            yield event
    """

if __name__ == "__main__":
    app.run()

## 내부적으로 무엇이 일어나나요?

`BedrockAgentCoreApp`을 사용하면 자동으로:

* 포트 8080에서 수신하는 HTTP 서버를 생성합니다
* 에이전트의 요구사항을 처리하기 위한 필수 `/invocations` 엔드포인트를 구현합니다
* 상태 확인을 위한 `/ping` 엔드포인트를 구현합니다 (비동기 에이전트에 매우 중요)
* 적절한 콘텐츠 유형과 응답 형식을 처리합니다
* AWS 표준에 따른 오류 처리를 관리합니다

### 2단계: 요구사항 파일 생성

Strands Agent 배포에 필요한 Python 종속성을 정의합니다.

**주요 종속성:**
- **aws-opentelemetry-distro**: AgentCore 관찰 가능성에 필요
- **strands-agents**: 핵심 Strands 프레임워크
- **bedrock-agentcore**: Runtime 통합

**관찰 가능성 통합:**
`aws-opentelemetry-distro` 라이브러리는 모니터링 및 추적을 위한 자동 계측을 활성화합니다. [AgentCore 관찰 가능성 가이드](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability-configure.html)에 문서화된 바와 같이, 컨테이너화된 환경(예: docker)에는 다음 명령을 추가해야 합니다:

```dockerfile
CMD ["opentelemetry-instrument", "python", "main.py"]
```

이 자동 계측 접근 방식은 포괄적인 관찰 가능성을 위해 OpenTelemetry SDK를 Python 경로에 자동으로 추가합니다.

In [ ]:
%%writefile requirements.txt
strands-agents
strands-agents-tools
bedrock-agentcore
bedrock-agentcore-starter-toolkit
boto3
aws-opentelemetry-distro>=0.10.0

### 3단계: AgentCore Runtime 구성

자동 리소스 생성을 통해 Bedrock AgentCore Runtime 구성을 설정합니다.

**생성된 아티팩트:**
이 단계에서는 필수 배포 파일을 생성합니다:
- **Dockerfile**: Strands Agent를 위한 컨테이너 구성
- **.dockerignore**: docker build 시 제외할 파일 목록
- **.bedrock_agentcore.yaml**: Runtime 배포 구성

bedrock_agentcore_starter_toolkit을 사용하여 에이전트를 구성할 때 opentelemetry 계측을 처리한다는 점에 유의하세요. 생성된 Dockerfile에는 다음이 포함됩니다:
```bash
CMD ["opentelemetry-instrument", "python", "runtime_agent_main.py"]
```

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
import boto3

region = boto3.session.Session().region_name
agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_getting_started"
)
response

### 4단계: AgentCore Runtime에 배포

컨테이너화 및 배포를 위해 AWS CodeBuild를 사용하여 배포 프로세스를 시작합니다.

**배포 프로세스:**
- Strands Agent의 컨테이너화된 버전을 빌드합니다
- 필요한 AWS 리소스를 생성합니다 (ECR 리포지토리, IAM 역할)
- 컨테이너 이미지를 Amazon ECR에 푸시합니다
- 관리형 자동 확장 서비스로 AgentCore Runtime에 배포합니다

In [ ]:
launch_result = agentcore_runtime.launch()

### 5단계: 배포 상태 확인

배포 진행 상황을 모니터링하고 런타임이 준비될 때까지 기다립니다:

In [ ]:
import time

print("AgentCore Runtime 상태 확인 중...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"초기 상태: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"상태: {status} - 대기 중...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime이 준비되었습니다!")
else:
    print(f"⚠ AgentCore Runtime 상태: {status}")
    
print(f"최종 상태: {status}")

agent_runtime_id = launch_result.agent_id
agent_runtime_arn = launch_result.agent_arn
ecr_repo_name = launch_result.ecr_uri.split('/')[1]
codebuild_name = launch_result.codebuild_id.split(':')[0]
print(f"Strands AgentCore Runtime ID: {agent_runtime_id}")
print(f"Strands AgentCore Runtime ARN: {agent_runtime_arn}")
print(f"Strands AgentCore Runtime용 ECR 리포지토리: {ecr_repo_name}")
print(f"Strands AgentCore Runtime용 CodeBuild 프로젝트: {codebuild_name}")

## 배포된 에이전트 테스트

다양한 프롬프트로 Bedrock AgentCore Runtime API를 통해 배포된 Strands Agent를 호출하여 모든 도구가 올바르게 작동하는지 확인합니다.

**agentcore_client.invoke_agent_runtime** 을 사용하여 agentcore runtime 에 배포된 agent 를 호출합니다. 

In [ ]:
import boto3
import json
import uuid

SESSION_ID = str(uuid.uuid4())

PROMPT = "홍콩의 날씨는 어떤가요? 온도를 화씨로 반환해주세요"

agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=boto3.session.Session().region_name
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_runtime_arn,
    qualifier="DEFAULT",
    runtimeSessionId=SESSION_ID, #대화 컨텍스트를 유지하고 더 나은 추적성을 위해 여러 요청에 걸쳐 동일한 세션 식별자를 제공
    payload=json.dumps({"prompt": PROMPT})
)

if "text/event-stream" in boto3_response.get("contentType", ""):
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                print(line)
else:
    events = []
    for event in boto3_response.get("response", []):
        print(event.decode('utf-8'))
        events.append(event)

### 대화 기록 테스트

동일한 세션 ID를 사용하여 여러 요청에 걸쳐 에이전트가 대화 컨텍스트를 유지하는지 확인합니다:

In [ ]:
PROMPT = "제가 무엇을 물어봤나요?"

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_runtime_arn,
    qualifier="DEFAULT",
    runtimeSessionId=SESSION_ID, #대화 컨텍스트를 유지하고 더 나은 추적성을 위해 여러 요청에 걸쳐 동일한 세션 식별자를 제공
    payload=json.dumps({"prompt": PROMPT})
)

if "text/event-stream" in boto3_response.get("contentType", ""):
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                print(line)
else:
    events = []
    for event in boto3_response.get("response", []):
        print(event.decode('utf-8'))
        events.append(event)

## 대화 기록 및 세션 관리

**Strands Agent 대화 관리:**
Strands Agent는 기본 `SlidingWindowConversationManager` 전략을 사용한 내장 대화 관리를 포함합니다. 

이는 세션 내에서 대화 컨텍스트를 자동으로 유지하지만 여러 세션에 걸쳐 지속되지는 않습니다.

**참조:** [Strands Agent 대화 관리](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/conversation-management/)

**AgentCore Runtime 세션 격리:**
- **세션 식별**: 애플리케이션에서 제공하는 고유한 `runtimeSessionId`로 식별되거나, `runtimeSessionId`가 비어있는 경우 첫 번째 호출에서 Runtime 자체에서 식별됩니다
- **세션 격리**: 완전히 격리된 CPU, 메모리 및 파일시스템 리소스를 가진 전용 microVM에서 실행됩니다
- **컨텍스트 보존**: 동일한 대화 내에서 여러 상호작용에 걸쳐 컨텍스트를 보존합니다
- **세션 지속시간**: 세션은 최대 8시간의 수명을 가지거나 비활성으로 인해 15분 후에 종료됩니다
- **자동 정리**: 세션 종료 후 전체 microVM이 종료되고 메모리가 정리됩니다

**참조:** [AgentCore Runtime 세션](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-sessions.html)


세션 지속시간을 넘어선 지속적인 메모리를 위해서는 장기 대화 저장 및 검색을 위한 **Bedrock AgentCore Memory**와 통합하세요.

**참조:** [AgentCore Memory: AI 에이전트에 메모리 추가](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/memory.html)

## Amazon CloudWatch의 AgentCore 관찰 가능성

### Bedrock AgentCore 관찰 가능성이란?

Amazon Bedrock AgentCore는 CloudWatch 및 X-Ray 통합을 통해 내장 관찰 가능성을 제공합니다. 이를 통해 에이전트 성능 모니터링, 요청 흐름 추적, 대화 패턴 분석이 가능합니다.

주요 기능:
- **세션 추적**: 개별 대화 모니터링
- **분산 추적**: 구성 요소 간 요청 추적
- **성능 메트릭**: 지연 시간, 처리량 및 오류율
- **스팬 분석**: 상세한 실행 분석

자세히 알아보기: [AgentCore 관찰 가능성 가이드](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html)

### 메인 대시보드 보기

2-3분 기다린 후 [CloudWatch 콘솔](https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#/gen-ai-observability/agent-core/agents) 에 접속하여 AgentCore 관찰 가능성 대시보드를 확인하세요:
![observability-main-dashboard.png](images/observability-main-dashboard.png)

### 세션 관리

`strands_getting_started`에서 **DEFAULT**를 클릭하여 세션 기록을 확인합니다. 이는 "홍콩의 날씨는 어떤가요?"와 "제가 무엇을 물어봤나요?"라는 두 개의 추적이 있는 테스트 세션을 보여줍니다:
![observability-session-list.png](images/observability-session-list.png)

### 세션 개요

세션을 선택하여 메트릭, 추적 타임라인 및 성능 데이터를 확인하세요:
![observability-trace-list.png](images/observability-trace-list.png)

### 추적 분석

임의의 추적을 클릭하여 상세한 실행 단계를 검토하세요:

**스팬 세부사항**:
![observability-trace-span-1.png](images/observability-trace-span-1.png)
![observability-trace-span-2.png](images/observability-trace-span-2.png)
![observability-trace-span-3.png](images/observability-trace-span-3.png)

## 리소스 정리 (선택사항)

배포된 리소스를 정리합니다:

In [ ]:
import boto3
import os

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr',region_name=region)
codebuild_client = boto3.client('codebuild',region_name=region)

try:
    print("AgentCore Runtime 삭제 중...")
    agentcore_control_client.delete_agent_runtime(agentRuntimeId=agent_runtime_id)
    print("✓ AgentCore Runtime 삭제가 시작되었습니다")

    print("ECR 리포지토리 삭제 중...")
    ecr_client.delete_repository(repositoryName=ecr_repo_name, force=True)
    print("✓ ECR 리포지토리가 삭제되었습니다")

    print("CodeBuild 프로젝트 삭제 중...")
    codebuild_client.delete_project(name=codebuild_name)
    print("✓ CodeBuild 프로젝트가 삭제되었습니다")

    print("Bedrock AgentCore 구성 파일 삭제 중...")
    os.remove(".bedrock_agentcore.yaml") 
    print("✓ .bedrock_agentcore.yaml이 삭제되었습니다")
except Exception as e:
    print(f"❌ 정리 중 오류 발생: {e}")
    print("일부 리소스를 수동으로 정리해야 할 수 있습니다.")

## 결론

이 실습에서 성공적으로:

- ✅ Strands Agent를 Bedrock AgentCore Runtime에 배포했습니다
- ✅ Amazon CloudWatch를 통한 관찰 가능성 및 모니터링을 구성했습니다
- ✅ AgentCore Runtime 환경에서 원격 에이전트를 호출하여 엔드투엔드 에이전트 기능을 테스트했습니다
- ✅ 세션 격리 및 자동 정리 기능을 탐색했습니다

## Strands Agent를 위한 AgentCore Runtime의 주요 이점

- **프로덕션 배포**: AI 에이전트를 위한 확장 가능한 관리형 인프라
- **포괄적인 관찰 가능성**: 내장 모니터링, 추적 및 디버깅 기능
- **세션 관리**: 자동 세션 격리 및 정리
- **엔터프라이즈 준비**: 프로덕션 워크로드를 위한 보안, 안정성 및 규정 준수 기능
